In [1]:
import pandas as pd
from IPython.display import display

In [15]:
import pandas as pd
from IPython.display import display
def analyze_ODECO_results(directory, log_file, output_file, print_flag=False):
    """
    Purpose:
    To take data from a bash script run of the ODECO solution that has been grepped and format the data
    before saving it into sheets in an Excel file grouped by tFinal and Re
    
    Keyword Arugments:
    directory - the directory the log_file is stored in and where the Excel file will be saved
    log_file - name of the grepped text file containing the results of a series of runs for the ODECO solution
    output_file - name of the Excel file to be saved
    print_flag - boolean flag used to turn on print statements of the resulting DataFrames (default is False)
    
    Returns:
    None
    """
    # Concatenate the directory with the filenames
    log_file = directory+log_file
    output_file = directory+output_file    
    # Get the data from the formatted csv
    data = pd.read_csv(log_file, sep=' ')
    display(data[:10])
    # Rename runtime to Runtime (seconds)
#     data = data.rename(columns={'runtime':'Runtime (seconds)'})
    # Create the ExcelWriter to send the information to xlsx
    writer = pd.ExcelWriter(output_file)
    
    # Loop over the choices for ending time (tFinal)
    for time in data.tFinal.unique():
        # Loop over the choices for the Reynolds number (Re)
        for Re in data.Re.unique():
            for numSens in data.numSensors.unique():
                # Get the information for a single pair of time and Re
                df = data[data.tFinal.eq(time) 
                          & data.Re.eq(Re) 
                          & data.numSensors.eq(numSens)].reset_index(drop=True)
                # Get the reduction in error from the increase  in spatial/temporal resolution
                df['alphaReduction'] = df['alphaErr2'].shift(1)/df['alphaErr2']
                df['uReduction'] = df['uErr2'].shift(1)/df['uErr2']
                # Write results to a sheet in output_file
                sheetname = 'tFinal='+str(time)+', Re='+str(Re)+', numSens='+str(numSens)
                df.to_excel(writer,
                           sheet_name=sheetname,
                           index=False,
                           columns=['Re','numSensors','nx','nSteps','ParameterSpace',
                                        'uErr2','uReduction',
                                       'alphaErr2','alphaReduction',
                                       'KKTRunTime','ErrorCheckRunTime','VTKRunTime','TotalTime'])
                workbook = writer.book
                # Base format used for all columns
                format_all = workbook.add_format()
                format_all.set_align('center')
                # Format the sheet
                sheet = writer.sheets[sheetname]
                width = 18
                sheet.set_column('A:M',width,format_all)
                # Format uReduction, alphaReduction
                format_reduction = workbook.add_format()
                format_reduction.set_align('center')
                format_reduction.set_num_format('0.00')            
                sheet.set_column('G:G',width,format_reduction)
                sheet.set_column('I:I',width,format_reduction)
                # Format the runtime columns
                format_time = workbook.add_format()
                format_time.set_num_format('0.00')
                sheet.set_column('J:M',width,format_time)
                if print_flag:
                    display(df)
    
    writer.save()
    writer.close()

# def analyze_results(log_file,output_file):
#     # Get the data from the formatted csv
#     data = pd.read_csv(log_file, sep=' ')
#     display(data.columns.values)
#     # Create the ExcelWriter to send the information to xlsx
#     writer = pd.ExcelWriter(output_file)
    
#     # Loop over the choices for ending time (tFinal)
#     for time in data.tFinal.unique():
#         # Loop over the choices for the Reynolds number (Re)
#         for Re in data.Re.unique():
#             for numSens in data.numSensors.unique():
#                 # Get the information for a single pair of time and Re
#                 df = data[data.tFinal.eq(time) 
#                           & data.Re.eq(Re) 
#                           & data.numSensors.eq(numSens)].reset_index(drop=True)
#                 # Get the reduction in error from the increase  in spatial/temporal resolution
#                 df['alphaReduction'] = df['alphaErr2'].shift(1)/df['alphaErr2']
#                 df['uReduction'] = df['uErr2'].shift(1)/df['uErr2']
#                 # Write results to a sheet in output_file
#                 sheetname = 'tFinal='+str(time)+', Re='+str(Re)+', numSens='+str(numSens)
#                 df.to_excel(writer,
#                            sheet_name=sheetname,
#                            index=False,
#                            columns=['Re','numSensors','nx','nSteps','ParameterSpace',
#                                         'uErr2','uReduction',
#                                        'alphaErr2','alphaReduction',
#                                        'KKTRunTime','ErrorCheckRunTime','VTKRunTime','TotalTime'])
#                 sheet = writer.sheets[sheetname]
#                 sheet.set_column('A:I',15)
#                 display(df)
    
#     writer.save()
#     writer.close()

In [16]:
# This go was for Re={10,100}, nx=24,36,48,60,72, tFinal=1,4
ODECO_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/ODECO/Log_Files/SingleParameterSpace/'
log_file = 'log_short_Re{1,20,40,60,80,100}_nxnt{24,36,48,60,72}_numSens{1,2,3,5,11}.txt'
output = 'ODECO_sensor_compare.xlsx'
analyze_ODECO_results(directory=ODECO_dir,log_file=log_file,output_file=output)

,tFinal,Re,numSensors,nx,nSteps,tol,ParameterSpace,alphaErr2,relAlphaErr2,uErr2,relUErr2,KKTRunTime,ErrorCheckRunTime,VTKRunTime,TotalTime
0,1,1,1,24,24,0.999,single,0.000366,0.004325,0.004335,0.004250,0.189725,1.95382,2.14859,NaN
1,1,1,2,24,24,0.999,single,0.000362,0.004273,0.004303,0.004217,0.156082,1.98647,2.14576,NaN
2,1,1,3,24,24,0.999,single,0.000289,0.003409,0.003738,0.003639,0.229698,1.97213,2.20663,NaN
3,1,1,5,24,24,0.999,single,0.000356,0.004206,0.004260,0.004173,0.317620,1.99447,2.31652,NaN
4,1,1,11,24,24,0.999,single,0.000286,0.003373,0.003718,0.003618,0.956193,1.96424,2.92259,NaN
5,1,1,1,36,36,0.999,single,0.000194,0.002297,0.002481,0.002460,0.302977,4.80128,5.10912,NaN
6,1,1,2,36,36,0.999,single,0.000197,0.002322,0.002505,0.002485,0.362661,4.87460,5.24565,NaN
7,1,1,3,36,36,0.999,single,0.000080,0.000945,0.001438,0.001402,0.380170,4.80175,5.19050,NaN
8,1,1,5,36,36,0.999,single,0.000214,0.002526,0.002681,0.002662,0.544255,4.89176,5.44490,NaN
9,1,1,11,36,36,0.999,single,0.000072,0.000848,0.001386,0.001349,1.393460,4.82169,6.22346,NaN
